In [1]:
import xlrd
import pandas as pd
import numpy as np
import json
import unidecode

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_excel('input/youth_survey_raw.xlsx')

# get question ids from columns
question_ids = [ c for c in df.columns if 'Q' in c ]

# copy row 0 to column names
df.columns = df.iloc[0]

# delete row 0
df = df.drop(0, axis=0)
df = df.applymap(str)


,M1: Questionnaire No,Project No,Project Name,M2: Governorate,M3: District,M7: Urbanity,M8: Date of interview,M8a: Day of interview,M9: Interview Validation,M11b: Date of Auditing,...,Q61_3_X: Other,S1: How old are you,S2: What is your marital status?,S3: Do you have any children?,"S4: (If ""yes""), How many?","S5: Including you, how many people, children, above and under 15 live in this household?",S6: What is the highest education level you completed?,"S7: Which one of the folowing income is close to your household monthly income, including all household members incomes live in or out this house?",S8: What is your job status today?,S11: Gender of respondent
1,79,132,Youth Survey,Ibb,Al-Sabrah,Rural,2017-05-11 00:00:00,Thu,Not validated,2017-05-11 00:00:00,...,nan,17,"Single, didn't not get married before",nan,nan,6,Didn't complete secondary school,"20,000 YER- 39,000 YER",Student,Male
2,101,132,Youth Survey,Ibb,Al-Makhader,Rural,2017-05-12 00:00:00,Fri,Accompanied interview,2017-05-12 00:00:00,...,nan,16,"Single, didn't not get married before",nan,nan,6,Completed intermediate school,"20,000 YER- 39,000 YER",Student,Male
3,172,132,Youth Survey,Abyan,Khanfar,Rural,2017-05-10 00:00:00,Wed,Re-contacted by person,2017-05-10 00:00:00,...,nan,25,Married,Yes,4,6,Illiterate (Can't read and write),"Less than 20,000 YER",Housewife,Female
4,1107,132,Youth Survey,Shabwah,Jardaan,Rural,2017-05-23 00:00:00,Tue,Not validated,2017-05-23 00:00:00,...,nan,19,Married,No,nan,9,Completed secondary school,"60,000 YER - 79,000 YER",Housewife,Female
5,1439,132,Youth Survey,ad-Dali,Qatabah,Rural,2017-05-10 00:00:00,Wed,Not validated,2017-05-10 00:00:00,...,nan,18,"Single, didn't not get married before",nan,nan,13,Didn't complete secondary school,"60,000 YER - 79,000 YER",Student,Male
6,917,132,Youth Survey,Hadhramawt,Al-Mukala City,Urban,2017-05-14 00:00:00,Sun,Not validated,2017-05-14 00:00:00,...,nan,16,"Single, didn't not get married before",nan,nan,4,Didn't complete secondary school,(DK),Student,Female
7,170,132,Youth Survey,Ibb,Yarim,Rural,2017-05-17 00:00:00,Wed,Not validated,2017-05-17 00:00:00,...,nan,24,Married,Yes,1,3,Completed elementary school,"20,000 YER- 39,000 YER",Housewife,Female
8,74,132,Youth Survey,Ibb,Al-Sabrah,Rural,2017-05-11 00:00:00,Thu,Not validated,2017-05-11 00:00:00,...,nan,25,Divorced,Yes,4,5,Illiterate (Can't read and write),"20,000 YER- 39,000 YER",Works in his/her own business,Female
9,1462,132,Youth Survey,ad-Dali,Al-Dhale,Urban,2017-05-14 00:00:00,Sun,Not validated,2017-05-14 00:00:00,...,nan,16,"Single, didn't not get married before",nan,nan,5,Didn't complete secondary school,"Less than 20,000 YER",Student,Male
10,1436,132,Youth Survey,ad-Dali,Qatabah,Rural,2017-05-10 00:00:00,Wed,Re-contacted by person,2017-05-10 00:00:00,...,nan,20,"Single, didn't not get married before",nan,nan,8,Completed secondary school,"20,000 YER- 39,000 YER","Unemployed, looking for job",Male


In [66]:
question_columns = [col.strip() for col in df.columns if col.startswith(('Q'))]
demographic_columns = [col.strip() for col in df.columns if col.startswith(('S'))]
demographic_columns

['S1: How old are you',
 'S2: What is your marital status?',
 'S3: Do you have any children?',
 'S4: (If "yes"), How many?',
 'S5: Including you, how many people, children, above and under 15 live in this household?',
 'S6: What is the highest education level you completed?',
 'S7: Which one of the folowing income is close to your household monthly income, including all household members incomes live in or out this house?',
 'S8: What is your job status today?',
 'S11: Gender of respondent']

In [134]:
df_ = df.filter(items=question_columns)
df_['governorate'] = df['M2: Governorate']
df_['district'] = df['District']
df_['urbanity'] = df['M7: Urbanity']
df_['age'] = df['S1: How old are you']
df_['marital_status'] = df['S2: What is your marital status?'].apply(lambda x: x.split(', ')[0])
df_['have_children'] = df['S3: Do you have any children?']
df_['job_status'] = df['S8: What is your job status today?'].apply(lambda x: x.replace('Unemployed, looking for job', 'Unemployed').replace('Works in his/her own business', 'Self-employed'))
df_['gender'] = df['S11: Gender of respondent']

df_.head()

,Q1: First I would like to ask you about your current situation / family situation. How would you rate it on a scale from 1 to 5?,"Q2: In general, what do you think about the current situation in Yemen, is it going in the right direction or in the wrong direction?",Q3: What is your status:,Q3x: Other,"Q4_1: What are the reasons you had to stop going to school, university, vocational training or lost your job? First answer",Q4_1_X :Other,"Q4_2: What are the reasons you had to stop going to school, university, vocational training or lost your job? Second answer",Q4_2_X :Other,Q5: How much time do you spend at school/university/work/vocational training?,Q6: How close is the school/university/work/vocational training you attend to your house?,...,Q61_2_X: Other,Q61_3: What are the Three Radio stations that you most listen? Third,Q61_3_X: Other,governorate,urbanity,age,marital_status,have_children,job_status,gender
1,Neither bad nor good,In a very bad direction,School student,nan,nan,nan,nan,nan,5-8 hours,0-1 km,...,nan,nan,nan,Ibb,Rural,17,Single,nan,Student,Male
2,It’s very bad,"3.Not in the right direction, not in the wrong...",School student,nan,nan,nan,nan,nan,1-4 hours,0-1 km,...,nan,nan,nan,Ibb,Rural,16,Single,nan,Student,Male
3,It’s very bad,In a very bad direction,I am a housewife,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,Abyan,Rural,25,Married,Yes,Housewife,Female
4,Neither bad nor good,"3.Not in the right direction, not in the wrong...",I am a housewife,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,Shabwah,Rural,19,Married,No,Housewife,Female
5,It’s somewhat good,In a very bad direction,School student,nan,nan,nan,nan,nan,1-4 hours,0-1 km,...,nan,nan,nan,ad-Dali,Rural,18,Single,nan,Student,Male


In [135]:
df_a_list = []
for q in question_columns:
    df_a = pd.DataFrame()
    
    
    
    q_ = unidecode.unidecode(q)
    qID = q_.split(':')[0]
    q_ = q_.split(':')[1].strip()
    
    
  
    if q_ != 'Other':
        df_a['governorate'] = df_['governorate']
        df_a['qID'] = qID
        df_a['question_raw'] = q_
        df_a['answer_raw'] = df_[q]
        df_a['urbanity'] = df_['urbanity']
        df_a['age'] = df_['age']
        df_a['marital_status'] = df_['marital_status']
        df_a['have_children'] = df_['have_children']
        df_a['job_status'] = df_['job_status']
        df_a['gender'] = df_['gender']
    
    
    

    df_a_list.append(df_a)

df_formatted = pd.concat(df_a_list)


df_formatted.head()
    

,governorate,qID,question_raw,answer_raw,urbanity,age,marital_status,have_children,job_status,gender
1,Ibb,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,17,Single,nan,Student,Male
2,Ibb,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,16,Single,nan,Student,Male
3,Abyan,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,25,Married,Yes,Housewife,Female
4,Shabwah,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,19,Married,No,Housewife,Female
5,ad-Dali,Q1,First I would like to ask you about your curre...,It’s somewhat good,Rural,18,Single,nan,Student,Male


In [133]:
# governorate > question > answer
df_gov_q_a = df_formatted[['governorate', 'question_raw', 'answer_raw']]
df_gov_q_a['count'] = ''
df_gov_q_a = df_gov_q_a.groupby(['governorate', 'question_raw', 'answer_raw']).count().reset_index()
df_gov_q_a.sort_values(['governorate', 'question_raw'])

/Users/mcenaneyc/Desktop/ypc-data/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,governorate,question_raw,answer_raw,count
0,Abyan,(If yes) How many times?,1,6
1,Abyan,(If yes) How many times?,2,5
2,Abyan,(If yes) How many times?,3,2
3,Abyan,(If yes) How many times?,4,1
4,Abyan,(If yes) How many times?,5,1
5,Abyan,(If yes) How many times?,6,1
6,Abyan,(If yes) How many times?,nan,14
7,Abyan,"According to your opinion, Are Political Parti...",Neither positive nor negative,1
8,Abyan,"According to your opinion, Are Political Parti...",Not active at all,3
9,Abyan,"According to your opinion, Are Political Parti...",nan,26
